# Fantasy Football Week One
## Data Collection

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time

In [2]:
%%capture

from tqdm import tqdm_notebook as tqdm
from tqdm import tnrange
tqdm().pandas()

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

### Passing Data Collection

In [10]:
passing_df = pd.DataFrame(columns = ['PLAYER', 'POSITION', 'TEAM', 'GP', 'ATT', 'CMP', 'PCT', 'YDS', 'YDS/G', 'LNG'
                                    'TD', 'INT', 'SCK', 'YDSL', 'RATE'])

In [11]:
url = 'https://www.cbssports.com/nfl/stats/player/passing/nfl/regular/all/'
html = requests.get(url)
html

<Response [200]>

In [12]:
soup = BeautifulSoup(html.content, 'html.parser')

In [48]:
container = soup.find('tbody')

In [110]:
container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[48].get_text()

'\n                    1\n                '

In [87]:
container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[0].get_text()

'\n                    1\n                '

In [89]:
container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[12].get_text()

'\n                    1\n                '

In [90]:
container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[24].get_text()

'\n                    1\n                '

In [63]:
def get_name(soup):
    container = soup.find('tbody')
    names = []
    for i in range(len(container.findAll('a', class_ = ''))):
        if i % 2 != 0:
            names.append(container.findAll('a', class_ = '')[i].get_text())
        else:
            continue
    return names

In [83]:
def get_position(soup):
    container = soup.find('tbody')
    positions = []
    for i in range(len(container.findAll('span', class_ = 'CellPlayerName-position'))):
        if i % 2 != 0:
            position = container.findAll('span', class_ = 'CellPlayerName-position')[i].get_text()
            position = position.replace('\n', '')
            position = position.replace(' ', '')
            positions.append(position)
        else:
            continue
    return positions

In [82]:
def get_team(soup):
    container = soup.find('tbody')
    teams = []
    for i in range(len(container.findAll('span', class_ = 'CellPlayerName-team'))):
        if i % 2 != 0:
            team = container.findAll('span', class_ = 'CellPlayerName-team')[i].get_text()
            team = team.replace('\n', '')
            team = team.replace(' ', '')
            teams.append(team)
        else:
            continue
    return teams

### 12 Stats

In [181]:
def get_first(soup):
    container = soup.find('tbody')
    first_col = []
    count = 0
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            first_col.append(val)
        
        count += 12    
    return first_col

In [182]:
def get_second(soup):
    container = soup.find('tbody')
    second_col = []
    count = 1
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            second_col.append(val)
        
        count += 12    
    return second_col

In [183]:
def get_third(soup):
    container = soup.find('tbody')
    third_col = []
    count = 2
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            third_col.append(val)
        
        count += 12    
    return third_col

In [184]:
def get_fourth(soup):
    container = soup.find('tbody')
    fourth_col = []
    count = 3
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            fourth_col.append(val)
        
        count += 12    
    return fourth_col

In [185]:
def get_fifth(soup):
    container = soup.find('tbody')
    fifth_col = []
    count = 4
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            fifth_col.append(val)
        
        count += 12    
    return fifth_col

In [186]:
def get_sixth(soup):
    container = soup.find('tbody')
    sixth_col = []
    count = 5
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            sixth_col.append(val)
        
        count += 12    
    return sixth_col

In [187]:
def get_seventh(soup):
    container = soup.find('tbody')
    seventh_col = []
    count = 6
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            seventh_col.append(val)
        
        count += 12    
    return seventh_col


In [188]:
def get_eighth(soup):
    container = soup.find('tbody')
    eighth_col = []
    count = 7
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            eighth_col.append(val)
        
        count += 12    
    return eighth_col

In [190]:
def get_ninth(soup):
    container = soup.find('tbody')
    ninth_col = []
    count = 8
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            ninth_col.append(val)
        
        count += 12    
    return ninth_col

In [191]:
def get_tenth(soup):
    container = soup.find('tbody')
    tenth_col = []
    count = 8
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            tenth_col.append(val)
        
        count += 12    
    return tenth_col

In [ ]:
def get_elev(soup):
    container = soup.find('tbody')
    ninth_col = []
    count = 8
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            ninth_col.append(inter)
        
        count += 12    
    return ninth_col